# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
   "com.snapchat.android_11.1.1.66-2097_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
   "com.snapchat.android_11.1.6.81-2099_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
     "com.instagram.android_160.0.0.25.132-246889068_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
     "com.instagram.android_160.0.0.25.132-246889061_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"

)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_160.0.0.25.132-246889068_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_160.0.0.25.132-246889061_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'SCREENSHOT_DETECTOR': 'CV5',
    'STORY_VIEWED_PLUGIN': 'YEh',
    'SNAP_MODEL': 'kmc',
    'APP_START_EXPERIMENT_MANAGER': 'B3f',
    'CBC_ENCRYPTION_ALGORITHM': 'q47'
}

In [6]:
named_m_decs = {
    'SCREENSHOT_DETECTED': MethodDec('CV5', 'a', 'CV5', 'BV5'),
    'MARK_STORY_AS_VIEWED': MethodDec('YEh', 'Y', 'Eke', 'qae'),
    'SNAP_MODEL_CONSTRUCTOR': MethodDec('kmc', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'cK5', 'long', 'boolean', 'java.lang.Long', 'long'),
    'FORCE_APP_DECK': MethodDec('B3f', 'f', 'Jc5'),
    'DECRYPT_MEDIA_STREAM': MethodDec('q47', 'y0', 'java.io.InputStream')
}

In [7]:
named_f_decs = {
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [9]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [10]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [11]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [12]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.1mw#A01() -> X.1nC#A01()
+ Found single candidate for Method. Considering it a match 
	X.0Rv#A00(android.content.Context) -> X.0S4#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0u(X.1p8, X.215) -> com.instagram.reels.fragment.ReelViewerFragment#A0u(X.1pQ, X.21O)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.inst

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [13]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

.. Considering ambiguous match by UniqueString as match with a majority of 0.91 (LX/7Ba; -> LX/6sw;)
+ Found single candidate for Class. Considering it a match! 
	LX/0Mk; -> LX/0Mk;
+ Found single candidate for Class. Considering it a match! 
	LX/0ip; -> LX/0j1;
+ Found single candidate for Class. Considering it a match! 
	LX/0pq; -> LX/0q2;
+ Found single candidate for Class. Considering it a match! 
	LX/0qN; -> LX/0qZ;
+ Found single candidate for Class. Considering it a match! 
	LX/12Y; -> LX/12k;
+ Found single candidate for Class. Considering it a match! 
	LX/12Z; -> LX/12l;
+ Found single candidate for Class. Considering it a match! 
	LX/12t; -> LX/135;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [14]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/7vy; -> LX/7nE;
+ Found single candidate for Class. Considering it a match! 
	LX/3x7; -> LX/3xe;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3tF; -> LX/3tm;
+ Found single candidate for Class. Considering it a match! 
	LX/5B2; -> LX/5QQ;
+ Found single candidate for Class. Considering it a match! 
	LX/7BE; -> LX/7uQ;
+ Found single candidate for Class. Considering it a match! 
	LX/0iy; -> LX/0jA;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/2s9; -> LX/2sa;
+ Found single candidate for Class. Considering it a match! 
	LX/1vx; -> LX/1wG;
* Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [15]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.26V#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2HS#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	X.7vy#<init>(X.1z5, X.85d, X.0Mk, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.7vP, X.7w3) -> X.7nE#<init>(X.1zO, X.7nl, X.0Mk, com.instagram.clips.intf.ClipsViewerConfig, android.view.View, X.7mX, X.7nI)
+ Found single candidate for Method. Considering it a match 
	X.3x7#onLongPress(android.view.MotionEvent) -> X.3xe#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1H9, androidx.fragment.app.FragmentActivity, X.0

## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [16]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0jt; -> LX/0k5;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1Xb; -> LX/1Xr;
+ Found single candidate for Class. Considering it a match! 
	LX/1yS; -> LX/1yl;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0qR; -> LX/0qd;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [17]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

74 / 46 Classes were matched
56 / 60 Methods were matched
37 / 39 Fields were matched

Matching Classes:
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1mw -> X.1nC
• X.0Rv -> X.0S4
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.46h -> X.47E
• X.0nR -> X.0nd
• X.1QT -> X.1Qj
• X.1gs -> X.1h8
• X.2A4 -> X.2AN
• X.2eK -> X.2el
• X.33v -> X.34R
• X.4Iy -> X.4IQ
• X.6U7 -> X.6Ri
• X.0Mk -> X.0Mk
• X.0ip -> X.0j1
• X.0pq -> X.0q2
• X.0qN -> X.0qZ
• X.12Y -> X.12k
• X.12Z -> X.12l
• X.12t -> X.135
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Kl -> X.1L1
• X.1Pu -> X.1QA
• X.1V0 

In [18]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.7nE', '<init>', 'X.1zO', 'X.7nl', 'X.0Mk', 'com.instagram.clips.intf.ClipsViewerConfig', 'android.view.View', 'X.7mX', 'X.7nI'),
    MethodDec('X.3xe', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1HM', 'androidx.fragment.app.FragmentActivity', 'X.0Mk', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagr

In [21]:
"""
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"
"""



c_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [22]:
%autoreload

c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mk)
No matching Method found for X.26V#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2HS#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.1td#A06
No matching Field found for X.1tg#A02


In [23]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = ClassDec("X.7nE")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3xe")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3tm")

    override val DM_HOLDER = ClassDec("X.5QQ")

    override val DM_REPLY = ClassDec("X.47E")

    override val DM_VOICEMESSAGE = ClassDec("X.5JZ")

    override val DMVIDEOPLAYER = ClassDec("X.7uQ")

    override val FEEDITEM = ClassDec("X.1VG")

    override val FRAGMENT = ClassDec("X.1QA")

    override v

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1L1"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.1zO"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3xe"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1Wy"])
    override val COMMENT_TEXT = VariableDec<String>("A0Y")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    @FieldClass(

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```